# Ejercicio 6.4.1 - Latex a Markdown

En este ejercicio vamos a hacer una versión simplificada de un traductor de texto en [Latex](https://es.wikipedia.org/wiki/LaTeX) a texto en [Markdown](https://es.wikipedia.org/wiki/Markdown). Para esto, solo consideraremos las siguientes transformaciones:

- Letras en negrita
- Letras en cursiva
- Listado de elementos

Si no estás familiarizado con Latex, a continuación te presentamos cómo se realizan las instrucciones listadas en este lenguaje:

![image](figs/fig-latex-instructions.png)


Si no estás familiarizado con Markdown, esta es la forma en que se realizan las mismas instrucciones anteriores:

![image](figs/fig-markdown-instructions.png)

Entonces, tu programa debe recibir un _string_ con un texto en formato Latex y debe imprimir el texto en formato Markdown. Ejemplos de la interacción del programa:

- Input del programa (texto en Latex):

```py
	"\\textit{Esto está en cursiva}. \\textbf{Esto en negrita}. "
```
	
- Output del programa (texto en Markdown):

```
    >>>
	_Esto está en cursiva_. **Esto en negrita**.
    >>>
```
	
- Input del programa (texto en Latex):

```py
	"Lo siguiente está itemizado: \\begin{itemize} \item Primer elemento \item Segundo elemento \\end{itemize} "
```
	
- Output del programa (texto en Markdown):
```
    >>>
	Lo siguiente está itemizado:
    - Primer elemento 
    - Segundo elemento
    >>>
```

- Nótese que el output anterior corresponde al siguiente *string*:
```py
    "Lo siguiente está itemizado: \n- Primer elemento \n- Segundo elemento"
```
	
Ten en cuenta lo siguiente:

- Fíjate en los ejemplos anteriores como se manejan los saltos de línea y como se escapan algunos caracteres (donde hay doble backslash).
-  Puedes asumir que el código que llega en Latex está bien escrito, por lo tanto, no tendrás problemas con las llaves. 
-  Para el caso de listar elementos, puedes considerar que la instrucción en latex te llegará sin saltos de línea. Pero al momento de transformarlo, ten en consideración que en markdown, los saltos de línea antes de cada guión son muy importantes. 
- Un texto a traducir puede contener una combinación de las tres instrucciones listadas y en cantidades arbitriarias. 
- También te recomendamos que empieces a poner en práctica lo aprendido en la sección anterior sobre funciones. 

Para la solución, implementamos primero las funciones para cada tipo de instrucción: negrita, cursiva y listado. En el caso de letras en negrita, tomamos le entregamos como parámetro el texto que se resalta, por lo que le agregamos los asteriscos correspondientes.

In [ ]:
def bold(text):
    return "**{}**".format(text)

En el caso de letras en cursiva, seguimos la misma lógica, pero en vez de los asteriscos, le agregamos un guión a cada lado.

In [ ]:
def italic(text):
    return "_{}_".format(text)

Para el caso de listar elementos, es un poco más complejo, ya que en el texto que queremos listar, tenemos que ir separando según ``\item``. Aquí se propone una manera de hacerlo, pero probablemente existen formas más eficientes de realizarlas una vez que se haya adquirido más conocimiento (por ejemplo, utilizando ``split()`` que se ve más adelante en la sección de listas, o también con expresiones regulares).

Notar que el parámetro ``text`` será un _string_ de este estilo:

```py
" \item Primer elemento \item Segundo elemento"
```

In [4]:
def itemize(text):
    # itemized text será el string que iremos formando con los elementos
    itemized_text = "" 
    # cada vez que listemos un elementos, lo eliminaremos del string, por lo que en text_actualizado iremos guardando el nuevo valor
    text_actualizado = text
    # podemos contar cuantos elementos tendremos contando los \item, para saber cuantas iteraciones haremos
    cantidad_elementos = text.count("\item")
   
    for i in range(cantidad_elementos):
        # vemos en qué posición comienza la primera instrucción \item
        index_item = text_actualizado.index("\item") 
        # con la variable anterior nos movemos 6 posiciones (el largo del string "\item" + 1) para saber donde comienza el elemento
        item_inicio = index_item + 6
        
        # si es que ya estamos en el último elemento, el fin del elemento será el fin del texto actualizado
        if i == cantidad_elementos - 1:
            item = text_actualizado[item_inicio:]
        # si aún nos quedan elementos por iterar, el elemento que estamos agregando llegará hasta el comienzo del siguiente \item
        else:
            item_fin = text_actualizado[item_inicio:].index("\item") + 7
            item = text_actualizado[item_inicio:item_fin]
            text_actualizado = text_actualizado[item_fin:]

        # agregamos el elemento con un salto de línea al comienzo, seguido se un guión
        itemized_text += "\n- {}".format(item)
        
    return itemized_text

Ahora realizamos nuestra función principal, que recibe el texto entero en formato latex y va buscando las diferentes instrucciones y actualizando el texto en una lógica más o menos similar  a la seguida en la función ``itemize``.

In [12]:
def latex_a_markdown(text_latex):
    markdown_text = text_latex

    while True:
        if "\\textbf{" in markdown_text:
            # Vemos donde comienza la instrucción
            index_instruccion = markdown_text.index("\\textbf{")
            # Vemos el inicio y el fin del texto en negrita
            bold_inicio = index_instruccion + 8 
            bold_fin = (markdown_text[index_instruccion:]).index("}") + len(markdown_text[:index_instruccion])
            # Usamos la función bold para pasarlo a formato markdown
            bold_markdown = bold(markdown_text[bold_inicio:bold_fin])
            # Actualizamos el texto en markdown
            markdown_text = markdown_text[:index_instruccion] + bold_markdown + markdown_text[bold_fin + 1:]
        
        elif "\\textit{" in markdown_text:
            # Seguimos la misma idea anterior
            index_instruccion = markdown_text.index("\\textit{")

            italic_inicio = index_instruccion + 8 
            italic_fin = markdown_text[index_instruccion:].index("}") + len(markdown_text[:index_instruccion])
            
            italic_markdown = italic(markdown_text[italic_inicio:italic_fin])
            
            markdown_text = markdown_text[:index_instruccion] + italic_markdown + markdown_text[italic_fin + 1:]

        elif "\\begin{itemize}" in markdown_text:
            # Vemos donde comienza y termina la instrucción
            index_instruccion = markdown_text.index("\\begin{itemize}")
            
            itemize_inicio = index_instruccion + 15
            itemize_fin = markdown_text.index("\\end{itemize}")

            itemized_markdown = itemize(markdown_text[itemize_inicio:itemize_fin])
            
            markdown_text = markdown_text[:index_instruccion] + itemized_markdown + markdown_text[itemize_fin + 14:]

        else:
            return markdown_text

Para probar nuestro programa, definimos algunos _string_ de prueba con combinaciones de instrucciones en latex e imprimimos lo que retorna la función ``latex_a_markdown()``:

In [16]:
bold_1 = "Hola \\textbf{esto está en negrita}. "
print(latex_a_markdown(bold_1))

Hola **esto está en negrita**. 


In [17]:
bold_2 = "Hola \\textbf{esto está en negrita}. \\textbf{Esto también}. "
print(latex_a_markdown(bold_2))

Hola **esto está en negrita**. **Esto también**. 


In [18]:
italic_1 = "Otro ejemplo: \\textit{esto está en cursiva}. "
print(latex_a_markdown(italic_1))

Otro ejemplo: _esto está en cursiva_. 


In [19]:
italic_2 = "Otro ejemplo: \\textit{esto está en cursiva}. \\textit{Esto también}. "
print(latex_a_markdown(italic_2))

Otro ejemplo: _esto está en cursiva_. _Esto también_. 


In [20]:
bold_italic = "\\textit{Esto está en cursiva}. \\textbf{Esto en negrita}. "
print(latex_a_markdown(bold_italic))

_Esto está en cursiva_. **Esto en negrita**. 


In [24]:
itemize_1 = "Lo siguiente está itemizado: \\begin{itemize} \item Primer elemento \item Segundo elemento \\end{itemize} "
print(latex_a_markdown(itemize_1))

Lo siguiente está itemizado: 
- Primer elemento 
- Segundo elemento 


In [26]:
itemize_2 = "Y ahora unos elementos: \\begin{itemize} \item Primer elemento \item Segundo elemento \\end{itemize}.\n Ahora otro: \\begin{itemize} \item Primer elemento \item Segundo elemento \\end{itemize}" 
print(latex_a_markdown(itemize_2))

Y ahora unos elementos: 
- Primer elemento 
- Segundo elemento 
 Ahora otro: 
- Primer elemento 
- Segundo elemento 


In [27]:
bold_italic_itemize = bold_1 + italic_1 + itemize_1
print(latex_a_markdown(bold_italic_itemize))

Hola **esto está en negrita**. Otro ejemplo: _esto está en cursiva_. Lo siguiente está itemizado: 
- Primer elemento 
- Segundo elemento 
